## Task 4 (Guided Exercise)

This notebook is part of Milestone 3, task 3 and is a guided exercise. I have put guidelines and helpful links (as comments) along with this notebook to take you through this.

In this exercise you will be using Spark's MLlib. The idea is to tune some hyperparameters of a Random Forest to find an optimum model. Once we know the optimum settings, we'll train a Random Forest in sklearn (task 4)and save it with joblib (task 5) (so that we can use it next week to deploy).

Here consider MLlib as another python package that you are using, like the scikit-learn. You will be seeing many scikit-learn similar classes and methods available in MLlib for various ML related tasks, you might also notice that some of them are not yet implimented in MLlib. What you write using pyspark package will be using the spark engine to run your code, and hence all the benefits of distributed computing what we discussed in class.

NOTE: Here whenever you use spark makes sure that you refer to the right documentation based on the version what you will be using. [Here](https://spark.apache.org/docs/) you can select the version of the spark and go to the correct documentation. In our case we are using spark 3.1.2, and here is the link to spark documetation that you can refer to,
- [MLlib Documentation](https://spark.apache.org/docs/3.1.2/ml-guide.html)
- [MLlib API Reference](https://spark.apache.org/docs/3.1.2/api/python/reference/pyspark.ml.html)

You may notice that there are RDD-based API and DataFrame-based (Main Guide) API available in the documentation. You want to focus on DataFrame based API as no one these days use RDD based API. We will discuss the difference in class.

Before you start this notebook make sure that you are using EMR jupyterHub and the kernal that you selected is PySpark.

## Import necessary libraries

In [1]:
from pyspark.ml import Pipeline
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.ml.feature import VectorAssembler, UnivariateFeatureSelector
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import RandomForestRegressor as sparkRFR
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import pandas as pd

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
6,application_1649815737766_0007,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Read the data

In [2]:
pandas_df = pd.read_csv("s3://mds-s3-12/output/ml_data_SYD.csv", index_col=0, storage_options=aws_credentials, parse_dates=True).dropna()
feature_cols = list(pandas_df.drop(columns="Observed").columns)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Preparing dataset for ML

In [3]:
# Load dataframe and coerce features into a single column called "Features"
# This is a requirement of MLlib
# Here we are converting your pandas dataframe to a spark dataframe, 
# Here "spark" is a spark session I will discuss this in our Wed class. 
# It is automatically created for you in this notebook.
# read more  here https://blog.knoldus.com/spark-createdataframe-vs-todf/
training = spark.createDataFrame(pandas_df)
assembler = VectorAssembler(inputCols=feature_cols, outputCol="Features")
training = assembler.transform(training).select("Features", "Observed")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Find best hyperparameter settings

Official Documentation of MLlib, Random forest regression [here](http://spark.apache.org/docs/3.0.1/ml-classification-regression.html#random-forest-regression).

Here we will be mainly using following classes and methods;

- [RandomForestRegressor](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.regression.RandomForestRegressor.html)
- [ParamGridBuilder](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.tuning.ParamGridBuilder.html)
    - addGrid
    - build
- [CrossValidator](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.tuning.CrossValidator.html)
    - fit

Use these parameters for coming up with ideal parameters, you could try more parameters, but make sure you have enough power to do it. But you are required to try only following parameters. This will take around 15 min on entire dataset....

    - Use numTrees as [10, 50,100]
    - maxDepth as [5, 10]
    - bootstrap as [False, True]
    - In the CrossValidator use evaluator to be RegressionEvaluator(labelCol="Observed")
    
***Additional reference:*** You can refer to [here](https://www.sparkitecture.io/machine-learning/regression/random-forest) and [here](https://www.silect.is/blog/random-forest-models-in-spark-ml/).
Some additional reading [here](https://projector-video-pdf-converter.datacamp.com/14989/chapter4.pdf)

In [5]:
#create model
random_forest = sparkRFR(labelCol='Observed')
#create grid search parameters
grid_param = ((ParamGridBuilder()
               .addGrid(random_forest.numTrees, [10, 50, 100])
               .addGrid(random_forest.maxDepth, [5, 10])
               .addGrid(random_forest.bootstrap, [False, True])
               )
             .build())

#create evaluator
evaluator = RegressionEvaluator(labelCol='Observed')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
#Apply evaluator and grid parameters to cross validator
cv = CrossValidator(estimator = random_forest,
                      estimatorParamMaps = gird_param,
                      evaluator = evaluator,
                      numFolds = 5)
final_model = cv.fit(training)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# Print run info
print("\nBest model")
print("==========")
print(f"\nCV Score: {min(final_model.avgMetrics):.2f}")
print(f"numTrees: {final_model.bestModel.getNumTrees}")
print(f"maxDepth: {final_model.bestModel.getMaxDepth()}")
print(f"bootstrap: {final_model.bestModel.getBootstrap()}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Best model

CV Score: 8.17
numTrees: 100
maxDepth: 5
bootstrap: False